# Оптимизировать производство



In [1]:
import numpy as np
import pandas as pd

from itertools import combinations

In [2]:
input_file = "input_data.xlsx"
output_file = "result.xlsx"

In [16]:
sheetname = "Orders"
df_orders = pd.read_excel(input_file, sheetname, nrows=None)
df_orders

,MatrixName,OrderSize
0,Орион - 600,734
1,Орион - 700,722
2,Орион - 800,1622
3,Весна - 600,218
4,Весна - 700,350
5,Весна - 800,820
6,Сафари,292
7,Сафари 2,352
8,Сатурн ПГ,42
9,Сатурн ПО,584


- NORDERS
- NLOT - кол-во мест для матриц пресса
- TCHANGE
- THEAT
- DIFF_TOT_TIME

In [4]:
NORDERS = df_orders.shape[0]
NLOT = 3

TCHANGE = 20
THEAT = 40

In [5]:
ORDERS = set(df_orders.MatrixName.values)
ORDERS

{'Весна - 600', 'Орион - 600', 'Орион - 700', 'Орион - 800'}

In [6]:
def convert_dataframe_to_dict(dataframe, key_columns, value_column):
    return (
        dataframe.loc[:, key_columns + [value_column]]
        .set_index(key_columns)
        .to_dict()[value_column]
    )

In [7]:
DURATION = convert_dataframe_to_dict(df_orders, ["MatrixName"], "OrderSize")
DURATION

{'Орион - 600': 734,
 'Орион - 700': 722,
 'Орион - 800': 1622,
 'Весна - 600': 218}

## Задача

lot_states = ['changing', 'heating', 'empty', 'busy']

In [8]:
class Machine:
    def __init__(self):
        self.idle_lot = None #
        self.lot = np.zeros(NLOT)
        
    def is_idle_lot(self):
        return (self.lot == 0).any()
    
    def get_idle_lot(self):
        return np.where(self.lot == 0)[0]
    
    def set_order_to_lot(self, order, lot_number):
        assert self.lot[lot_number] == 0
        self.lot[lot_number] = order
        
    def get_first_done_lot(self):
        return self.lot[self.lot != 0].argmin()
    
    def get_done_value(self):
        return self.lot[self.get_first_done_lot()]
    
    def run(self):
        self.lot[self.lot != 0] -= self.get_done_value()
        

In [9]:
m = Machine()
m.lot

array([0., 0., 0.])

In [10]:
def get_subdict(dictionary, keys):
    return {k:dictionary[k] for k in keys}

In [11]:
def get_min_item(dictionary):
    return min(dictionary.items(), key=lambda x: x[1])

In [12]:
def reduce_values(dictionary, keys, value):
    for k in keys:
        dictionary[k] -= value
    return dictionary

In [13]:
def get_rest_orders(dictionary):
    num_empty_lots = 0
    rest_orders = []
    for k, v in dictionary.items():
        if v == 0:
            num_empty_lots += 1
        else:
            rest_orders.append(k)
    return rest_orders, num_empty_lots 

In [14]:
tmin = 1e15
seqmin = None
for comb_orders in combinations(DURATION.keys(), 3):
    current = DURATION.copy()
    
    t0 = 0
    prod_sequence = [comb_orders]
    min_lot, min_order = get_min_item(get_subdict(current, comb_orders))
    
    t0 += min_order + TCHANGE
    current = reduce_values(current, comb_orders, min_order)
    
    rest_orders, num_empty_lots = get_rest_orders(current)
    
    print(current, rest_orders, num_empty_lots)

{'Орион - 600': 12, 'Орион - 700': 0, 'Орион - 800': 900, 'Весна - 600': 218} ['Орион - 600', 'Орион - 800', 'Весна - 600'] 1
{'Орион - 600': 516, 'Орион - 700': 504, 'Орион - 800': 1622, 'Весна - 600': 0} ['Орион - 600', 'Орион - 700', 'Орион - 800'] 1
{'Орион - 600': 516, 'Орион - 700': 722, 'Орион - 800': 1404, 'Весна - 600': 0} ['Орион - 600', 'Орион - 700', 'Орион - 800'] 1
{'Орион - 600': 734, 'Орион - 700': 504, 'Орион - 800': 1404, 'Весна - 600': 0} ['Орион - 600', 'Орион - 700', 'Орион - 800'] 1


In [15]:
get_min_item({1:0, 2: 0, 3:1})

(1, 0)